# LWLR

In [1]:
# from google.colab import files
# up = files.upload()

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class LocallyWeightedLinearRegression:
    def __init__(self, tau=1.0):
        self.tau = tau

    def local_weight(self, x0, X):
        # Calculate weights using Gaussian kernel
        weights = np.exp(-np.sum((X - x0) ** 2, axis=1) / (2 * self.tau ** 2))
        return np.diag(weights)

    def predict(self, X, y, x_query):
        # Calculate weights for the query point
        W = self.local_weight(x_query, X)

        # Closed-form solution for weighted linear regression
        XTWX = X.T @ W @ X
        XTWy = X.T @ W @ y
        theta = np.linalg.inv(XTWX) @ XTWy

        # Predict the value
        return x_query @ theta

    def predict_all(self, X, y, X_queries):
        # Predict the outputs for multiple query points
        predictions = []
        for x_query in X_queries:
            predictions.append(self.predict(X, y, x_query))
        return np.array(predictions)


# Main function
if __name__ == "__main__":
    # Import dataset
    df = pd.read_csv('data.csv')  # Replace with your file path

    # define X , y
    X = df[['A', 'B', 'C']].values  # Adjust columns as per your dataset
    y = df['T'].values

    # Spliting
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Add bias term (intercept) to features
    X_train_bias = np.hstack((np.ones((X_train_scaled.shape[0], 1)), X_train_scaled))
    X_test_bias = np.hstack((np.ones((X_test_scaled.shape[0], 1)), X_test_scaled))

    # Fit the LWLR model
    tau = 1.0  # Bandwidth parameter
    lwlr = LocallyWeightedLinearRegression(tau=tau)
    print("Model initialized with tau =", tau)

    # Predict values on test set
    y_pred = lwlr.predict_all(X_train_bias, y_train, X_test_bias)

    # Evaluate the model (R² score)
    r2 = r2_score(y_test, y_pred)
    print("R² score:", r2)

Model initialized with tau = 1.0
R² score: 0.9249316117356162
